In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [251]:
class Value:
    
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._prev = set(_children)
        self._op = _op
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, value):
        out = Value(self.data + value.data, (self, value), '+')
        return out
    
    def __mul__(self, value):
        out = Value(self.data * value.data, (self, value), '*')
        return out
    
    def __sub__(self, value):
        out = Value(self.data - value.data, (self, value), '-')
        return out
    
    def __truediv__(self, value):
        out = Value(self.data / value.data, (self, value), '/')
        return out
    
    def tanh(self):
        n = self.data
        t = (math.exp(2*n) - 1) / (math.exp(2*n) + 1)
        out = Value(t, (self, ), 'tanh')
        return out
    
    def relu(self):
        if self.data > 0:
            return Value(self.data, (self, ), 'relu')
        else:
            return Value(0.0, (self, ), 'relu')
    
    def backward(self, earlier_derivative=None):
        
        if len(self._prev) == 1:
            child1 = list(self._prev)[0]
            
            if self._op == 'tanh':
                n = child1.data
                t = (math.exp(2*n) - 1) / (math.exp(2*n) + 1)
                child1.grad += (1 - (t * t))
                
            if self._op == 'relu':
                if self.data > 0:
                    self.grad = 1
                else:
                    self.grad = 0
                
            if earlier_derivative:
                child1.grad *= earlier_derivative
            else:
                self.grad = 1
                
            if len(child1._prev) > 0:
                child1.backward(child1.grad)
                
        elif len(self._prev) == 2:
            child1, child2 = self._prev
            
            if self._op == '+':
                child1.grad += 1
                child2.grad += 1
                
            elif self._op == '*':
                child1.grad += child2.data
                child2.grad += child1.data
                
            elif self._op == '-':
                child1.grad += 1
                child2.grad += -1
                
            elif self._op == '/':
                child2.grad += 1 / child1.data
                child1.grad += - child2.data / (child1.data * child1.data)
                
            #elif self._op == '**':
            #    child1._grad 
            
            if earlier_derivative:
                child1.grad *= earlier_derivative
                child2.grad *= earlier_derivative
            else:
                self.grad = 1
            
            if len(child1._prev) > 0:
                child1.backward(child1.grad)
                
            if len(child2._prev) > 0:
                child2.backward(child2.grad)
                

                
    def zero_grad(self):
        
        if len(self._prev) > 0:
            child1, child2 = self._prev
            child1.grad, child2.grad = 0, 0
            
            if len(child1._prev) > 0:
                child1.zero_grad()
                
            if len(child2._prev) > 0:
                child2.zero_grad()

In [252]:
x1 = Value(2.0)
x2 = Value(0.0)

w1 = Value(-3.0)
w2 = Value(1.0)

b = Value(6.8813735870195432)

x1w1 = x1*w1
x2w2 = x2*w2
x1w1x2w2 = x1w1 + x2w2
n = x1w1x2w2 + b
o = n.tanh()

In [253]:
o.backward()


In [254]:
n

Value(data=0.8813735870195432)

In [258]:
x1w1x2w2

Value(data=-6.0)

In [172]:
a = Value(3)
b = Value(6)

In [173]:
c = a / b

In [174]:
c

Value(data=0.5)

In [175]:
c.backward()

In [176]:
a.grad

0.16666666666666666

In [177]:
b.grad

-0.08333333333333333

In [121]:
-0.12*25

-3.0

In [122]:
-3/25

-0.12